<a href="https://colab.research.google.com/github/MUYang99/NOx-Time-Series-Prediction-Based-on-Deep-Learning/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.optimizers import adam_v2
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy import stats, arange

In [ ]:
### Import & process data
df = pd.read_excel('d2.xlsx')
df.columns = ['Date', 'NOx']
df = df.dropna(subset=['NOx'])
data = df[df.Date < '20151231']


In [154]:
# Import data
def load_data() -> pd.DataFrame:

  df = pd.read_excel('d2.xlsx')
  df.columns = ['Date', 'NOx']
  df = df.dropna(subset=['NOx'])

  return df

In [155]:
# MinMaxScaler 
# For speeding up the model fitting and improving the accuracy
def minmaxscaler(data: pd.DataFrame) -> pd.DataFrame:

  # nox = data.NOx.values
  nox = data.values
  nox = nox.reshape(len(nox), 1)
  nox = scaler.fit_transform(nox)
  nox = nox.reshape(len(nox),)
  # data['NOx'] = nox
  # data.columns = ['', 'NOx']

  return nox

In [156]:
# Build data set
def build_dataset(data):

  #data = data.drop(["Date"], axis=1)
  X, Y = [], []
  for i in range(data.shape[0]-n_in-n_out+1):
    # X.append(np.array(data.iloc[i:i+n_in]))
    # Y.append(np.array(data.iloc[i+n_in:i+n_in+n_out]))
    X.append(np.array(data[i:i+n_in]))
    Y.append(np.array(data[i+n_in:i+n_in+n_out]))

  return np.array(X), np.array(Y)

In [157]:
# Split the train and validation datasets
def split_data(x, y):
  
  x_train = x[:-n_val-n_out+1]
  x_val = x[-n_val:]
  y_train = y[:-n_val-n_out+1]
  y_val = y[-n_val:]

  return x_train, y_train, x_val, y_val

In [158]:
# Contruct the LSTM
def build_lstm():

  model = Sequential()
  # model.add(LSTM(n_neuron, input_shape=(n_in, n_features)))
  model.add(LSTM(n_neuron, input_shape=(n_in, n_features)))
  #model.add(LSTM(200, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(n_out))
  # model.add(Activation("sigmoid"))
  model.compile(optimizer=adam_v2.Adam(learning_rate=1e-4), loss='mae')

  return model

In [159]:
class RocAucMetricCallback(Callback):
    def __init__(self, predict_batch_size=1024):
        super(RocAucMetricCallback, self).__init__()
        self.predict_batch_size = predict_batch_size
 
    def on_batch_begin(self, batch, logs={}):
        pass
 
    def on_batch_end(self, batch, logs={}):
        pass
 
    def on_train_begin(self, logs={}):
        if not ('val_roc_auc' in self.params['metrics']):
            self.params['metrics'].append('val_roc_auc')
 
    def on_train_end(self, logs={}):
        pass
 
    def on_epoch_begin(self, epoch, logs={}):
        pass
 
    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc'] = float('-inf')
        if (self.validation_data):
            logs['roc_auc'] = roc_auc_score(self.validation_data[1], self.model.predict(self.validation_data[0], batch_size=self.predict_batch_size))
            print('ROC_AUC - epoch:%d - score:%.6f' % (epoch + 1, logs['roc_auc']))

In [160]:
# Train the model
def model_fit(x_train, y_train, x_val, y_val):
    
    model = build_lstm()
    
    my_callbacks = [
        RocAucMetricCallback(),
        EarlyStopping(monitor='roc_auc', patience=20, verbose=2, mode='max')
    ]
    history = model.fit(x_train, y_train, batch_size=batchsize, epochs=n_epochs, verbose=1, validation_data=(x_val, y_val))
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    plt.show()
    
    return model

In [161]:
n_in = 672
n_out = 144
n_features = 1
n_val = 1

n_epochs = 30
batchsize = 128
n_neuron = 150

In [162]:
data = load_data()
data = data[data.Date < '20151231']
d1 = data.drop(["Date"], axis=1)

In [167]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = minmaxscaler(data)

In [168]:
data_copy = data.copy()
x, y = build_dataset(data_copy)
x_train, y_train, x_val, y_val = split_data(x, y)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
y_val = np.reshape(y_val, (y_val.shape[0], y_val.shape[1], 1))

(15716, 672, 1)
(15716, 144, 1)
(1, 672, 1)
(1, 144, 1)


In [169]:
model = build_lstm()
model = model_fit(x_train, y_train, x_val, y_val)

Epoch 1/30
123/123 [==============================] - 20s 149ms/step - loss: 0.1743 - val_loss: 0.1245
Epoch 2/30
123/123 [==============================] - 18s 146ms/step - loss: 0.1458 - val_loss: 0.1231
Epoch 3/30
123/123 [==============================] - 18s 146ms/step - loss: 0.1431 - val_loss: 0.1164
Epoch 4/30
123/123 [==============================] - 18s 145ms/step - loss: 0.1381 - val_loss: 0.1040
Epoch 5/30
123/123 [==============================] - 18s 147ms/step - loss: 0.1317 - val_loss: 0.1042
Epoch 6/30
123/123 [==============================] - 18s 146ms/step - loss: 0.1292 - val_loss: 0.1035
Epoch 7/30
123/123 [==============================] - 18s 145ms/step - loss: 0.1279 - val_loss: 0.1055
Epoch 8/30
123/123 [==============================] - 18s 146ms/step - loss: 0.1272 - val_loss: 0.1047
Epoch 9/30
123/123 [==============================] - 18s 146ms/step - loss: 0.1265 - val_loss: 0.1017
Epoch 10/30
123/123 [==============================] - 18s 146ms/step - l

KeyboardInterrupt: ignored

In [ ]:
# Validation & visulization
predict = model.predict(x_val)
validation = scaler.inverse_transform(predict)[0]
validation

In [ ]:
y_val = y_val.reshape(1,n_out)
true = scaler.inverse_transform(y_val)[0]
true

In [ ]:
x = [x for x in range(n_out)]
fig, ax = plt.subplots(figsize=(15,5), dpi = 300)
ax.plot(x, validation, linewidth=2.0, label = "predict")
ax.plot(x, true, linewidth=2.0, label = "true")
ax.legend(loc=2);
plt.grid(linestyle='-.')
plt.show()

In [ ]:
MSE = mean_squared_error(true,validation)
RMSE = np.sqrt(MSE)
print('Test RMSE: %.3f' %RMSE)
MAE = mean_absolute_error(true,validation)
print('Test MAE: %.3f' %MAE)

In [163]:
##################################### Box-plot #####################################
# fig = plt.figure(figsize = (10,6))
# ax1 = fig.add_subplot(2,1,1)
# color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
# d1.plot.box(vert=False, grid = True,color = color,ax = ax1,label = 'NOx data')

# s = data.describe()
# print(s)
# print('------')
# q1 = s['NOx'][4]
# q3 = s['NOx'][6]
# iqr = q3 - q1
# min = q1 - 1.5*iqr
# max = q3 + 1.5*iqr
# print('IQR：%.3f，min：%.3f，max：%.3f' % (iqr,min,max))

# ax2 = fig.add_subplot(3,1,3)
# error = d1['NOx'][(d1['NOx'] < min) | (d1['NOx'] > max)]
# data_c = d1['NOx'][(d1['NOx'] >= min) & (d1['NOx'] <= max)]
# print('The number of outliers: %i' % len(error)) 
# plt.scatter(data_c.index,data_c,color = 'k',marker='.',alpha = 0.3)
# plt.scatter(error.index,error,color = 'r',marker='.',alpha = 0.5)
# plt.grid()

In [ ]:
##################################### Normal-distribution(3 sigma) #####################################
# u = d1['NOx'].mean() 
# std = float(d1['NOx'].std())
# stats.kstest(d1['NOx'], 'norm', (u, std))
# print('Mean：%.3f，Std：%.3f' % (u, std))

# sigma = 3
# fig = plt.figure(figsize = (10,6))
# d1.plot(kind = 'kde',grid = True,style = '-k',title = 'Density curve')
# plt.axvline(sigma*std,color='r',linestyle="--",alpha=0.8) 
# plt.axvline(-sigma*std,color='r',linestyle="--",alpha=0.8)

# error = d1['NOx'][np.abs(d1['NOx'] - u) > sigma*std]
# data_c = d1['NOx'][np.abs(d1['NOx'] - u) <= sigma*std]
# print('The number of outliers: %i' % len(error))
# ax1 = fig.add_subplot(3,1,1)
# plt.scatter(data_c.index,data_c,color = 'k',marker='.',alpha = 0.3)
# plt.scatter(error.index,error,color = 'r',marker='.',alpha = 0.5)
# plt.grid()